In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import clumpiness
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xgboost

In [3]:
def create_metadata():
    """
    For interest here is how the metadata was constructed from the individual files, otherwise this is stored in 
    its completed form in metadata.csv
    """
    # Bailer-Jones distance
    bj = pd.read_csv('Bailer-Jones.tsv', delimiter='\t')
    # Entry in TIC (TESS Input Catalogue)
    tic = pd.read_csv('TICv8_iota_Draconis.csv', comment='#')
    # Gaia data
    gaia = pd.read_csv('1594239954087O-result.csv')

    # Put it all together
    # id r_est, r_error, astrometric_chi2_al, astrometric_n_good_obs_al, astrometric_excess_noise, kmag, bp_rp, numax, phot_g_mean_mag, ra, dec
    distance = bj['rest'].values.item()
    # Not needed when predicting, this is needed for training chunked versions of full Kepler data to "smear" distance distribution
    # When training need to add 'r_lo' and 'r_hi' columnds from Bailer-Jones table as well.
    distance_error = 0
    # Gaia information
    excess = gaia['astrometric_excess_noise'].values.item()
    kmag = tic['Kmag'].values.item()
    # Add numax which isn't ever used but made it easier to keep track of if it was also passed through feature extractor
    numax = np.nan
    
    # Position on the sky
    ra = gaia['ra'].values.item()
    dec = gaia['dec'].values.item()
    
    metadata = pd.DataFrame(data=np.array(['1377591', distance, distance_error, excess, kmag, numax, ra, dec])[None,:],
                            columns=['ID', 'distance', 'distance_error', 'astrometric_excess_noise', 'Kmag', 'numax', 'ra', 'dec'])

    metadata.to_csv('iotaDrac_metadata.csv', index=False)
    
#create_metadata()

In [4]:
# Dataset length is only 52 days but we round up here to 80 days to use that trained model, not that it really makes much difference
NDAYS = 80
model_params = clumpiness.config.retrieve_model_parameters(dataset_length=NDAYS)

In [5]:
# Load in model
bst = xgboost.Booster(model_params)
#bst.load_model('../Clumpiness-package/Code/-1random.model')
bst.load_model('../../models/'+str(NDAYS)+'.model')
best_iter = np.loadtxt('../../models/best_iter_'+str(NDAYS)+'.txt')
print(f"Best iter: {best_iter}")

Best iter: 301.0


In [6]:
# Import metadata
metadata = pd.read_csv('iotaDrac_metadata.csv')
metadata

,ID,distance,distance_error,astrometric_excess_noise,Kmag,numax,ra,dec
0,1377591,31.64925,0,1.829604,0.671,NaN,231.232323,58.96614


In [14]:
# Set up data ready for feature computation
data_file = '1377591.dat'
star = clumpiness.Dataset_regular.Dataset('1377591', data_file)
# Read in the dataset with the correct cadence
# ndays is set to -1 because we don't want to cut down the timeseries into smaller chunks
star.read_timeseries(dt=120.0)
# Do data preparation
star.processing()

In [15]:
# Set up feature class
feat = clumpiness.features.Features(star)

# Compute features
zs = feat.compute_zero_crossings()
var = feat.compute_var()
hoc, _ = feat.compute_higher_order_crossings(5)
mc = feat.compute_variance_change()
fill = feat.compute_fill()
# Compute absolute magnitude
abs_mag, mu0, Ak, good_flag, distance = feat.compute_abs_K_mag(metadata['Kmag'].values.item(), metadata['distance'].values.item(), 
                                                               metadata['distance_error'].values.item(), 
                                                               metadata['ra'].values.item(), metadata['dec'].values.item(), 
                                                               metadata['astrometric_excess_noise'].values.item())

In [16]:
# Print out features just to visually inspect them and make sure they are sensible
print(zs, np.log10(var), hoc, np.log10(mc), fill)

0.08342928855927177 2.4457237417062245 1.3875889929036926 1.8148800494317134 0.9260631335931732


In [17]:
# Set up array for prediction
predict = xgboost.DMatrix(np.array([var, zs, hoc, abs_mag, mc]).reshape(1,-1))

In [18]:
# Make the prediction
preds = bst.predict(predict, ntree_limit=best_iter).squeeze()
print(preds)

[3.5798427e-04 1.0115444e-03 5.6203318e-01 4.3573317e-01 8.6411112e-04]


In [19]:
# Combine the predictions into RGB, RC and "KOI"
RGB_prob = preds[0] + preds[1] + preds[2]
RC_prob = preds[3]
KOI_prob = preds[4]

In [20]:
print(f"This star has a RGB probability of {RGB_prob}, RC probability of {RC_prob} and probability of being neither of {KOI_prob}")

This star has a RGB probability of 0.5634027123451233, RC probability of 0.4357331693172455 and probability of being neither of 0.0008641111198812723
